# Huggingface 🤗 NLP Course

Neste curso serão abordados tópicos sobre Natural Language Processing (NLP) usando as bibliotecas do ecossistem a do [hugging face](https://huggingface.co/), como 🤗 Transformers, 🤗 Datasets, 🤗 Tokenizers, and 🤗 Accelerate — assim como Hugging Face Hub.

## Visão Geral
- Os capítulos 1 a 4 fornecem uma introdução aos principais conceitos da biblioteca 🤗 Transformers. Ao final desta parte do curso, você estará familiarizado com o funcionamento dos modelos do Transformer e saberá como usar um modelo do Hugging Face Hub, ajustá-lo em um conjunto de dados e compartilhar seus resultados no Hub!
- Os capítulos 5 a 8 ensinam o básico de 🤗 Conjuntos de dados e 🤗 Tokenizers antes de mergulhar nas tarefas clássicas de PNL. Ao final desta parte, você será capaz de resolver sozinho os problemas mais comuns da PNL.
- Os capítulos 9 a 12 vão além da PNL e exploram como os modelos Transformer podem ser usados para lidar com tarefas de processamento de fala e visão computacional. Ao longo do caminho, você aprenderá como criar e compartilhar demonstrações de seus modelos e otimizá-los para ambientes de produção. Ao final desta parte, você estará pronto para aplicar 🤗 Transformers a (quase) qualquer problema de aprendizado de máquina!

# 1. Modelos Transformers

## O que é NLP?
NLP é um campo da linguística e da Aprendizagem de Máquina (ML) focada em entender tudo relacionado a linguagem humana. O objetivo das tarefas de NLP não é apenas entender palavras soltas individualmente, mas ser capaz de entender o contexto dessas palavras.

A seguir uma lista de tarefas comuns de NLP, com alguns exemplos:

- Classificação de sentenças completas: Capturar o sentimento de uma revisão, detectar se um email é spam, determinar se a sentença é gramaticalmente correta ou onde duas sentenças são logicamente relacionadas ou não
- Classificação de cada palavra em uma sentença: Identificar os componentes gramaticais de uma sentença (substantivo, verbo, adjetivo), ou as entidades nomeadas (pessoa, local, organização)
- Geração de conteúdo textual: Completar um trecho com autogeração textual, preenchendo as lacunas em um texto com palavras mascaradas
Extrair uma resposta de um texto: Dada uma pergunta e um contexto, extrair a resposta baseada na informação passada no contexto
- Gerar uma nova sentença a partir de uma entrada de texto: Traduzir um texto para outro idioma, resumi-lo

NLP não se limita ao texto escrito. Também engloba desafios complexos nos campos de reconhecimento de discurso e visão computacional, tal como a geração de transcrição de uma amostra de áudio ou a descrição de uma imagem.


## Por que isso é desafiador?
Os computadores não processam a informação da mesma forma que os seres humanos. Por exemplo, quando nós lemos a sentença “Estou com fome”, nós podemos facilmente entender seu significado. Similarmente, dada duas sentenças como “Estou com fome” e “Estou triste”, nós somos capazes de facilmente determinar quão similares elas são. Para modelos de Aprendizagem de Máquina (ML), tarefas como essas são mais difíceis. O texto precisa ser processado de um modo que possibilite o modelo aprender por ele. E porque a linguagem é complexa, nós precisamos pensar cuidadosamente sobre como esse processamento tem que ser feito. Tem se feito muita pesquisa sobre como representar um texto e nós iremos observar alguns desses métodos no próximo capítulo.



## Transformers, o que eles podem fazer?

O objeto mais básico na biblioteca 🤗 Transformers é a função pipeline() . Ela conecta o modelo com seus passos necessários de pré e pós-processamento, permitindo-nos a diretamente inserir qualquer texto e obter uma resposta inteligível.


Há três principais passos envolvidos quando você passa algum texto para um pipeline:

O texto é pré-processado para um formato que o modelo consiga entender.
As entradas (inputs) pré-processados são passadas para o modelo.
As predições do modelo são pós-processadas, para que então você consiga atribuir sentido a elas.
Alguns dos pipelines disponíveis atualmente, são:

- feature-extraction (pega a representação vetorial do texto)
- fill-mask (preenchimento de máscara)
- ner (reconhecimento de entidades nomeadas)
- question-answering (responder perguntas)
- sentiment-analysis (análise de sentimentos)
- summarization (sumarização)
- text-generation (geração de texto)
- translation (tradução)
- zero-shot-classification (classificação “zero-shot”)

In [5]:
# !pip install datasets evaluate transformers[sentencepiece]

In [6]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [7]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

### Classificação Zero-shot
Nós começaremos abordando uma tarefa mais desafiadora da qual nós precisamos classificar texto que não tenham sido rotulados. Esse é um cenário comum nos projetos reais porque anotar texto geralmente consome bastante do nosso tempo e requer expertise no domínio. Para esse caso, o pipeline zero-shot-classification é muito poderoso: permite você especificar quais rótulos usar para a classificação, desse modo você não precisa “confiar” nos rótulos dos modelos pré-treinados. Você já viu como um modelo pode classificar uma sentença como positiva ou negativa usando esses dois rótulos - mas também pode ser classificado usando qualquer outro conjunto de rótulos que você quiser. Esse pipeline é chamado de zero-shot porque você não precisa fazer o ajuste fino do modelo nos dados que você o utiliza.

In [8]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445989489555359, 0.11197412759065628, 0.04342695698142052]}

### Geração de Texto
Agora vamos ver como usar um pipeline para gerar uma porção de texto. A principal ideia aqui é que você coloque um pedaço de texto e o modelo irá autocompletá-lo ao gerar o texto restante. Isso é similar ao recurso de predição textual que é encontrado em inúmeros celulares. A geração de texto envolve aleatoriedade, então é normal se você não obter o mesmo resultado obtido mostrado abaixo. Você pode controlar quão diferentes sequências são geradas com o argumento num_return_sequences e o tamanho total da saída de texto (output) com o argumento max_length.

In [9]:
from transformers import pipeline

generator = pipeline("text-generation")
generator(
    "In this course, we will teach you how to"
)  # nesse curso, nós te mostraremos como você

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to integrate Android into Android Studio, which will help achieve high performance and maintainability in your project. With this course (which is available as an MP3 download), we will work together with you and your'}]

In [10]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to get involved in public and private businesses and the benefits of these new jobs and what you can do to'},
 {'generated_text': 'In this course, we will teach you how to use JavaScript. This content will be hosted in the browser and on a mobile device.\n\n\n'}]

### Preenchimento de máscara (*Mask filling*)
O próximo pipeline que você irá testar é o fill-mask. A ideia dessa tarefa é preencher os espaços em branco com um texto dado. O argumento top_k controla quantas possibilidades você quer que sejam geradas. Note que aqui o modelo preenche com uma palavra <máscara> especial, que é frequentemente referida como mask token. Outros modelos de preenchimento de máscara podem ter diferentes mask tokens, então é sempre bom verificar uma palavra máscara apropriada quando explorar outros modelos. Um modo de checar isso é olhando para a palavra máscara usada no widget.

In [11]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.19619806110858917,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052723944187164,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

### Reconhecimento de entidades nomeadas
Reconhecimento de Entidades Nomeadas (NER) é uma tarefa onde o modelo tem de achar quais partes do texto correspondem a entidades como pessoas, locais, organizações. Vamos olhar em um exemplo. Aqui o modelo corretamente identificou que Sylvain é uma pessoa (PER), Hugging Face é uma organização (ORG), e Brooklyn é um local (LOC).

Nós passamos a opção grouped_entities=True na criação da função do pipelina para dize-lo para reagrupar juntos as partes da sentença que correspondem à mesma entidade: aqui o modelo agrupou corretamente “Hugging” e “Face” como única organização, ainda que o mesmo nome consista em múltiplas palavras. Na verdade, como veremos no próximo capítulo, o pré-processamento até mesmo divide algumas palavras em partes menores. Por exemplo, Sylvain é dividido em 4 pedaços: S, ##yl, ##va, e ##in. No passo de pós-processamento, o pipeline satisfatoriamente reagrupa esses pedaços.

In [12]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### question-answering
O pipeline question-answering responde perguntas usando informações dado um contexto. Note que o pipeline funciona através da extração da informação dado um contexto; não gera uma resposta.

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.6949767470359802, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

### Sumarização
Sumarização é uma tarefa de reduzir um texto em um texto menor enquanto pega toda (ou boa parte) dos aspectos importantes do texto referenciado. Como a geração de texto, você pode especificar o tamanho máximo max_length ou mínimo min_length para o resultado

In [14]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' The number of engineering graduates in the United States has declined in recent years . China and India graduate six and eight times as many traditional engineers as the U.S. does . Rapidly developing economies such as China continue to encourage and advance the teaching of engineering . There are declining offerings in engineering subjects dealing with infrastructure, infrastructure, the environment, and related issues .'}]

### Tradução
Para tradução, você pode usar o modelo default se você der um par de idiomas no nome da tarefa (tal como "translation_en_to_fr", para traduzir inglês para francês), mas a maneira mais fácil é pegar o moddelo que você quiser e usa-lo no Model Hub. Aqui nós iremos tentar traduzir do Francês para o Inglês. Como a geração de texto e a sumarização, você pode especificar o tamanho máximo max_length e mínimo min_length para o resultado.

In [15]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'This course is produced by Hugging Face.'}]

## Como transformers funciona ?

Em linhas gerais, eles podem ser agrupados em três categorias:

- GPT-like (também chamados de modelos Transformers auto-regressivos)
- BERT-like (também chamados de modelos Transformers auto-codificadores)
- BART/T5-like (também chamados de modelos Transformers sequence-to-sequence)

### Transformers são modelos de linguagem
Todos os modelos de Transformer mencionados acima (GPT, BERT, BART, T5, etc.) foram treinados como ```modelos de linguagem (langauge models)```. Isso significa que eles foram treinados em grandes quantidades de texto bruto de forma auto-supervisionada. O aprendizado autossupervisionado é um tipo de treinamento no qual o objetivo é calculado automaticamente a partir das entradas do modelo. Isso significa que os humanos não são necessários para rotular os dados!

Este tipo de modelo desenvolve uma compreensão estatística da linguagem em que foi treinado, mas não é muito útil para tarefas práticas específicas. Por causa disso, o modelo geral pré-treinado passa por um processo chamado ```transfer learning```. Durante esse processo, o modelo é ajustado de maneira supervisionada - ou seja, usando rótulos anotados por humanos - em uma determinada tarefa.

Um exemplo de tarefa é prever a próxima palavra em uma frase depois de ler as n palavras anteriores. Isso é chamado de ```modelagem de linguagem causal (causal language modeling)``` porque a saída depende das entradas passadas e presentes, mas não das futuras.

### Transformers são modelos grandes
Além de alguns outliers (como o DistilBERT), a estratégia geral para obter melhor desempenho é aumentar os tamanhos dos modelos, bem como a quantidade de dados em que são pré-treinados.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/model_parameters.png">

Pré-treinamento é o ato de treinar um modelo do zero: os pesos são inicializados aleatoriamente e o treinamento começa sem nenhum conhecimento prévio.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/pretraining.svg">

Esse pré-treinamento geralmente é feito em grandes quantidades de dados. Portanto, requer um corpus de dados muito grande e o treinamento pode levar várias semanas.

```Ajuste fino (fine tuning)```, por outro lado, é o treinamento feito **após** um modelo ter sido pré-treinado. Para realizar o ajuste fino, primeiro você adquire um modelo de linguagem pré-treinado e, em seguida, realiza treinamento adicional com um conjunto de dados específico para sua tarefa. Espere - por que não simplesmente treinar diretamente para a tarefa final? Existem algumas razões:

- O modelo pré-treinado já foi treinado em um conjunto de dados que possui algumas semelhanças com o conjunto de dados de ajuste fino. O processo de ajuste fino é, portanto, capaz de aproveitar o conhecimento adquirido pelo modelo inicial durante o pré-treinamento (por exemplo, com problemas de NLP, o modelo pré-treinado terá algum tipo de compreensão estatística da linguagem que você está usando para sua tarefa).
- Como o modelo pré-treinado já foi treinado com muitos dados, o ajuste fino requer muito menos dados para obter resultados decentes.
Pela mesma razão, a quantidade de tempo e recursos necessários para obter bons resultados são muito menores.

Por exemplo, pode-se aproveitar um modelo pré-treinado treinado no idioma inglês e, em seguida, ajustá-lo em um corpus arXiv, resultando em um modelo baseado em ciência/pesquisa. O ajuste fino exigirá apenas uma quantidade limitada de dados: o conhecimento que o modelo pré-treinado adquiriu é “transferido”, daí o termo ```transfer learning```.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/finetuning.svg">

O ajuste fino de um modelo de linguagem é mais barato do que o pré-treinamento, tanto em tempo quanto em dinheiro. Portanto, acarreta menores custos de tempo, dados, financeiros e ambientais. Também é mais rápido e fácil iterar diferentes esquemas de ajuste fino, pois o treinamento é menos restritivo do que um pré-treinamento completo.

Este processo também alcançará melhores resultados do que treinar do zero (a menos que você tenha muitos dados), e é por isso que você deve sempre tentar aproveitar um modelo pré-treinado – um que seja o mais próximo possível da tarefa que você tem em mãos – e ajustar isto.

### Arquitetura geral

O modelo transformer é composto principalmente por dois blocos:

- Encoder/Codificador (esquerda): O codificador recebe uma entrada e constrói uma representação dela (seus recursos). Isso significa que o modelo é otimizado para adquirir entendimento a partir da entrada.
- Decoder/Decodificador (direita): O decodificador usa a representação (recursos) do codificador junto com outras entradas para gerar uma sequência alvo. Isso significa que o modelo está otimizado para gerar resultados.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers_blocks.svg">


Cada uma dessas partes pode ser usada de forma independente, dependendo da tarefa:

- Encoder-only models: bons para tarefas que exigem compreensão da entrada, como **classificação de frases** e **reconhecimento de entidade nomeada**.
- Decoder-only models: bons para tarefas generativas, como **geração de texto**.
- Encoder-decoder models ou sequence-to-sequence models: bons para tarefas generativas que requerem uma entrada, como **tradução** ou **resumo**.

### Camadas de Atenção

Uma característica importante dos modelos Transformer é que eles são construídos com camadas especiais chamadas **camadas de atenção**. Na verdade, o título do artigo que apresenta a arquitetura do Transformer era _“Attention is all you need”. Esta camada dirá ao modelo para prestar atenção específica a certas palavras da frase que você passou (e mais ou menos ignorar as outras) ao lidar com a representação de cada palavra.

Para contextualizar isso, considere a tarefa de traduzir um texto do inglês para o francês. Dada a entrada “Você gosta deste curso”, um modelo de tradução precisará também atender à palavra adjacente “Você” para obter a tradução adequada para a palavra “gosto”, porque em francês o verbo “gosto” é conjugado de forma diferente dependendo de o sujeito. O resto da frase, porém, não é útil para a tradução dessa palavra. Na mesma linha, ao traduzir “isto” o modelo também precisará prestar atenção à palavra “curso”, porque “isto” é traduzido de forma diferente dependendo se o substantivo associado é masculino ou feminino. Novamente, as outras palavras da frase não importarão para a tradução de “isto”. Com frases mais complexas (e regras gramaticais mais complexas), o modelo precisaria prestar atenção especial às palavras que possam aparecer mais distantes na frase para traduzir corretamente cada palavra.

O mesmo conceito se aplica a qualquer tarefa associada à linguagem natural: uma palavra por si só tem um significado, mas esse significado é profundamente afetado pelo contexto, que pode ser qualquer outra palavra (ou palavras) antes ou depois da palavra em estudo.

### A arquitetura original

A arquitetura Transformer foi originalmente projetada para tradução. Durante o treinamento, o **codificador recebe entradas (frases) em um determinado idioma**, enquanto o **decodificador recebe as mesmas frases no idioma de destino desejado**. No codificador, **as camadas de atenção podem usar todas as palavras em uma frase** (já que, como acabamos de ver, a tradução de uma determinada palavra pode ser dependente do que está depois e antes dela na frase). **O decodificador, no entanto, funciona sequencialmente e só pode prestar atenção nas palavras da frase que ele já traduziu** (portanto, apenas as palavras anteriores à palavra que está sendo gerada no momento). Por exemplo, quando previmos as três primeiras palavras do alvo traduzido, as entregamos ao decodificador que então usa todas as entradas do codificador para tentar prever a quarta palavra.

Para acelerar as coisas durante o treinamento (quando o modelo tem acesso às frases alvo), o decodificador é alimentado com todo o alvo, mas não é permitido usar palavras futuras (se teve acesso à palavra na posição 2 ao tentar prever a palavra na posição 2, o problema não seria muito difícil!). Por exemplo, ao tentar prever a quarta palavra, a camada de atenção só terá acesso às palavras nas posições 1 a 3.

A arquitetura original do Transformer ficou assim, com o codificador à esquerda e o decodificador à direita:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers.svg">

Observe que **a primeira camada de atenção em um bloco decodificador presta atenção a todas as entradas (passadas) do decodificador**, mas a **segunda camada de atenção usa a saída do codificador**. Ele pode, assim, acessar toda a frase de entrada para melhor prever a palavra atual. Isso é muito útil, pois diferentes idiomas podem ter regras gramaticais que colocam as palavras em ordens diferentes, ou algum contexto fornecido posteriormente na frase pode ser útil para determinar a melhor tradução de uma determinada palavra.

A _máscara de atenção_ (attention mask) também pode ser usada no codificador/decodificador para evitar que o modelo preste atenção a algumas palavras especiais - por exemplo, a palavra de preenchimento especial usada para fazer com que todas as entradas tenham o mesmo comprimento ao agrupar frases.

### Arquiteturas vs. checkpoints
- **Arquitetura**: Este é o esqueleto do modelo — a definição de cada camada e cada operação que acontece dentro do modelo.
- **Checkpoints**: Esses são os pesos que serão carregados em uma determinada arquitetura.
- **Modelos**: Este é um termo abrangente que não é tão preciso quanto “arquitetura” ou “checkpoint”: pode significar ambos. Este curso especificará arquitetura ou checkpoint quando for necessário reduzir a ambiguidade.
Por exemplo, BERT é uma arquitetura enquanto bert-base-cased, um conjunto de pesos treinados pela equipe do Google para a primeira versão do BERT, é um checkpoint. No entanto, pode-se dizer “o modelo BERT” e “o modelo bert-base-cased“.

## Encoder Models

Os modelos de **encoder** usam apenas o encoder de um modelo Transformer. Em cada estágio, as camadas de atenção podem acessar todas as palavras da frase inicial. Cada palavra, é representada numericamente através de um vetor contextualizado gerado pelo encoder. Essa a contextualização é criada através do mecanismo de **self-attention**. O vetor gerado é chamado de vetor de features ou tensor de features, e ele segue o tamanho (dimensão) do vetor é definida pela arquitetura do modelo, no caso do BERT base, o tamanho é de 768. 


Esses modelos geralmente são caracterizados como tendo atenção “bidirecional” e são frequentemente chamados de ```auto-encoding models```.

O pré-treinamento desses modelos geralmente gira em torno de corromper de alguma forma uma determinada frase (por exemplo, mascarando palavras aleatórias nela) e encarregando o modelo de encontrar ou reconstruir a frase inicial.

Os modelos de codificador são mais adequados para tarefas que exigem uma compreensão da sentença completa, como classificação de sentença, reconhecimento de entidade nomeada (e, mais geralmente, classificação de palavras) e resposta extrativa de perguntas.

Modelos encoder são bons em extrair informações contextuais. Esses modelos sãoa dequados para atividades como:
- Sequence Classification
- Question Answering
- Masked Langauge modeling
- NLU: Natural language understanding

Os representantes desta família de modelos incluem:
- ALBERT
- BERT
- DistilBERT
- ELECTRA
- RoBERTa

## Decoder Models

Os modelos de decodificador usam apenas o decodificador de um modelo Transformer. Em cada etapa, para uma determinada palavra, as camadas de atenção só podem acessar as palavras posicionadas antes dela na frase. Esses modelos geralmente são chamados de modelos ```auto-regressive models```. O pré-treinamento de modelos de decodificadores geralmente gira em torno de prever a próxima palavra na frase.

Os inputs passados no decoder também criam uma representação numérica das palavras, de forma contextualizada. Mas diferente do encoder, no encoder o self attention é chamado de **masked self attention**, e essas representações numéricas das palavras são criadas somente das palavras anteriores até a atual, **não são usadas todas as palavras**. Todas as palavras da direta são ocultas (mascaradas). Os modelos encoder **não** são bidirecionais, e sim, **unidirecional**. 

Esses modelos são mais adequados para tarefas que envolvem geração de texto. Como
- Causal tasks; geração de sequencias
- NLG: Natural language generation

Os representantes desta família de modelos incluem:
- CTRL
- GPT
- GPT-2
- Transformer XL

## Sequence-to-sequence models

Modelos **encoder-decoder** (também chamados de modelos **sequence-to-sequence**) usam ambas as partes da arquitetura Transformer. Em cada estágio, as camadas de atenção do encoder podem acessar todas as palavras da frase inicial, enquanto as camadas de atenção do decoder podem acessar apenas as palavras posicionadas antes de uma determinada palavra na entrada.

O pré-treinamento desses modelos pode ser feito usando os objetivos dos modelos de codificador ou decodificador, mas geralmente envolve algo um pouco mais complexo. Por exemplo, T5 é pré-treinado substituindo trechos aleatórios de texto (que podem conter várias palavras) por uma única palavra especial de máscara, e o objetivo é prever o texto que esta palavra de máscara substitui.

No encoder-decoder, além de receber os inputs inicialmente, ele também recebe os outputs do encoder, na segunda camada de atenção. Quando não é passado uma sequencia inicial, são atribuṕidos valores que indica o início de uma sequencia. O encoder recebe a sequencia como entrada, calcula uma previsão e gera a representação numérica. Depois, ele envia a sequencia encodada para o decoder, e ele usa essa entrada junto com sua entrada de sequencia do decoder e tenta decodificar a sequencia. Assim, atuando de forma autoregressiva prevendo a próxima palavra a partir das palavras atuais. 

Os modelos **seq2seq** são capazes de atuar em tarefas de:
- Tradução
- Summarization

Os representantes desta família de modelos incluem:
- BART
- mBART
- Marian
- T5

# Usando 🤗 Transformers

A biblioteca 🤗 Transformers foi criada para fornecer uma API única através do qual qualquer modelo Transformer possa ser carregado, treinado e salvo. As principais características da biblioteca são:

- Fácil de usar: Baixar, carregar e usar um modelo de processamento natural de linguagem (PNL) de última geração para inferência pode ser feito em apenas duas linhas de código
- Flexibilidade: Em sua essência, todos os modelos são uma simples classe PyTorch nn.Module ou TensorFlow tf.keras.Model e podem ser tratadas como qualquer outro modelo em seus respectivos frameworks de machine learning (ML).
- Simplicidade: Quase nenhuma abstração é feita em toda a biblioteca. O “Tudo em um arquivo” é um conceito principal: o “passe para frente” de um modelo é inteiramente definido em um único arquivo, de modo que o código em si seja compreensível e hackeável.

Esta última característica torna 🤗 Transformers bem diferente de outras bibliotecas ML que modelos e/ou configurações são compartilhados entre arquivos; em vez disso, cada modelo tem suas próprias camadas. Além de tornar os modelos mais acessíveis e compreensíveis, isto permite que você experimente facilmente um modelo sem afetar outros.

A função pipeline agrupa três processos: tokenizer, model e post processing
- Tokenizer: É responsável por criar a representação numérica do texto de entrada, e, depois retornar ao texto quando necessário. Recebe a entrada de texto e divide em palavras e palavras, adiciona tokens especiais (de início da sentença [CLS] e final [SEP]). Então, o tokenizer associa cada token ao seu id exclusivo do vocabulário do modelo pré-treinado.
- Model: Recebe as representações numéricas do texto de entrada, e retorna um output Logit* da tarefa executada. 
- Post processing: A etapa de pós processamento recebe a saída do model e aplica uma função de ativação, como softmax, para normalizar os valores de logit em probabilidade. O retorno do pós processamento é uma lista com dois valores (no caso da classificação), em que um representa a classe negativa e outro positiva. Para ver qual valor representa é qual, basta utilizar ```model.config.id2label```.

***Logit*** valores que não estão diretamente relacionados a probabilidades. Eles representam a saída da última camada da rede neural antes de aplicar uma função de ativação, como a função sigmoid ou softmax. Quanto maior o logit para uma classe, mais a rede neural está "pensando" que a imagem pertence a essa classe.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg">

Exemplo

```python
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)
```

tem como resultado:

```python

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]
 
```

Aprofundando cada etapa...

### Pré-processamento com o tokenizer

Como outras redes neurais, os modelos do Transformer não podem processar texto bruto diretamente, portanto, a primeira etapa do nosso pipeline é converter as entradas de texto em números que o modelo possa entender. Para isso utilizamos um tokenizer, que será responsável por:

- Dividir a entrada em palavras, subpalavras ou símbolos (como pontuação) que são chamados de tokens
Mapeando cada token para um número inteiro
- Adicionando entradas adicionais que podem ser úteis para o modelo
- Todo esse pré-processamento precisa ser feito exatamente da mesma maneira que quando o modelo foi pré-treinado, então primeiro precisamos baixar essas informações do **Model Hub**. Para fazer isso, usamos a classe ```AutoTokenizer``` e seu método ```from_pretrained()```. Aqui, instanciando o modelo no objeto ```checkpoint```, ele irá buscar automaticamente os dados associados ao tokenizer do modelo e armazená-los em cache (portanto, ele só será baixado na primeira vez que você executar o código abaixo).

Como o modelo padrão do pipeline de análise de sentimento é ```distilbert-base-uncased-finetuned-sst-2-english```, executamos o seguinte:

```python
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
```

Assim que tivermos o tokenizer, podemos passar diretamente nossas frases para ele e receberemos de volta um dicionário que está pronto para alimentar nosso modelo! A única coisa que falta fazer é converter a lista de identificações de entrada em tensores.

Você pode usar 🤗 Transformers sem ter que se preocupar com qual estrutura ML é usada como backend; pode ser PyTorch ou TensorFlow, ou Flax para alguns modelos. Entretanto, os Transformers só aceitam **tensores** como entrada. Os tensores são estruturas de dados fundamentais em matemática e no campo da aprendizagem de máquina, incluindo deep learning. Eles podem ser pensados como uma generalização de vetores e matrizes, que são estruturas unidimensionais e bidimensionais, respectivamente. Em termos simples, você pode imaginar tensores como recipientes que podem armazenar dados em diferentes formatos. Cada elemento dentro de um tensor é identificado por um conjunto de índices. Vou dar alguns exemplos para ilustrar:

1. **Tensor 0D (Escalar)**: Este é o tensor mais simples, representando um único número. Por exemplo, a temperatura de um dia (30 graus Celsius) pode ser armazenada em um tensor 0D.

2. **Tensor 1D (Vetor)**: Este é um tensor unidimensional, como uma lista de números. Um exemplo é a sequência de preços de ações ao longo de vários dias.

3. **Tensor 2D (Matriz)**: Aqui, você tem uma estrutura de tabela com linhas e colunas, como uma planilha. Uma imagem em tons de cinza pode ser representada como uma matriz, onde cada elemento representa a intensidade do pixel.

4. **Tensor 3D**: Este tipo de tensor é tridimensional e pode ser usado para representar dados como um conjunto de imagens coloridas, onde uma dimensão adiciona a informação de cor (por exemplo, vermelho, verde, azul).

5. **Tensor ND (Tensor Multidimensional)**: Tensores podem ter mais de três dimensões, tornando-os adequados para uma variedade de aplicações complexas. Por exemplo, vídeos podem ser representados como tensores 4D, onde as dimensões são largura, altura, tempo e canais de cor.

Os tensores são a base para armazenar e manipular dados em deep learning e matemática aplicada. Eles são usados para representar entradas, saídas e parâmetros em redes neurais, permitindo que modelos de aprendizado de máquina processem informações em várias dimensões e realizem operações matemáticas complexas. Portanto, tensores desempenham um papel crucial na análise de dados, visão computacional, processamento de linguagem natural e muitas outras aplicações.

Para especificar o tipo de tensores que queremos recuperar (PyTorch, TensorFlow ou NumPy), utilizamos o argumento ```return_tensors```:

```python
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
```
O parametro ```padding```foi utilizado para criar matrizes do mesmo tamanho para frases com tamanhos diferentes, enquanto que o ```truncation``` foi usado pra limitar o tamanho da frase para o tamanho que o modelo suporta.

```python
{
    'input_ids': tensor([
        [  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172, 2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,     0,     0,     0,     0,     0,     0]
    ]), 
    'attention_mask': tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ])
}
```
Os resultados como tensores é um dicionário contendo duas chaves, ```input_ids``` e ```attention_mask```. O ```input_ids``` contém duas linhas de inteiros (uma para cada frase) que são os identificadores únicos dos tokens em cada frase. Jś os valores da chave ```attention_mask``` mostra onde foi aplicado o mecanismo de atenção, como as duas frases tem tamanhos diferentes, onde foi inserido o ```padding``` não foi aplicado o mecanismo de atenção.

### Etapa do modelo

Podemos baixar nosso modelo pré-treinado da mesma forma que fizemos com nosso tokenizer. 🤗 Transformers fornece uma classe AutoModel que também tem um método from_pretrained():

```python
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
```

Neste trecho de código, fizemos o download do mesmo checkpoint que usamos anteriormente em nosso pipeline (já deveria estar em cache) e instanciamos um modelo com ele.

Esta arquitetura contém apenas o módulo base do Transformer: dadas algumas entradas, ele produz o que chamaremos de hidden states (estados ocultos), também conhecidos como features (características). Para cada modelo de entrada, recuperaremos um vetor de alta dimensionalidade representando a compreensão contextual dessa entrada pelo Transformer*.

A saída vetorial pelo módulo do Transformer é geralmente grande. Geralmente tem três dimensões:

- Tamanho do lote (Batch size): O número de sequências processadas de cada vez (2 em nosso exemplo).
- Tamanho da sequencia (Sequence length): O comprimento da representação numérica da sequência (16 em nosso exemplo).
- Tamanho oculto (Hidden size): A dimensão vetorial de cada modelo de entrada.

Diz-se que é “de alta dimensionalidade” por causa do último valor. O tamanho oculto pode ser muito grande (768 é comum para modelos menores, e em modelos maiores isto pode chegar a 3072 ou mais).

Podemos ver isso se alimentarmos os inputs que pré-processamos para nosso modelo:

```python
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])
```

Observe que as saídas dos 🤗 Transformer se comportam como ‘tuplas nomeadas’ (namedtuple) ou dicionários. Você pode acessar os elementos por atributos (como fizemos) ou por chave (outputs["last_hidden_state"]), ou mesmo por índice se você souber exatamente onde o que está procurando (outputs[0]).

As heads do modelo usam o vetor de alta dimensionalidade dos hidden states como entrada e os projetam em uma dimensão diferente. Eles são geralmente compostos de uma ou algumas camadas lineares:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg">

A saída do Transformer é enviada diretamente para a head do modelo a ser processado.

Neste diagrama, o modelo é representado por sua camada de embeddings (vetores) e pelas camadas subsequentes. A camada de embeddings converte cada ID de entrada na entrada tokenizada em um vetor que representa o token associado. As camadas subsequentes manipulam esses vetores usando o mecanismo de atenção para produzir a representação final das sentenças.

Há muitas arquiteturas diferentes disponíveis no 🤗 Transformers, com cada uma projetada em torno de uma tarefa específica. Aqui está uma lista por algumas destas tarefas:

* Model (recuperar os hidden states)
* ForCausalLM
* ForMaskedLM
* ForMultipleChoice
* ForQuestionAnswering
* ForSequenceClassification
* ForTokenClassification
e outros 🤗

Para nosso exemplo, precisaremos de um modelo com uma head de classificação em sequencia (para poder classificar as sentenças como positivas ou negativas). Portanto, não utilizaremos a classe ```AutoModel```, mas sim, a classe ```AutoModelForSequenceClassification```:

```python
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
```

Agora se observarmos o tamanho dos nossos inputs, a dimensionalidade será muito menor: a head do modelo toma como entrada os vetores de alta dimensionalidade que vimos anteriormente, e os vetores de saída contendo dois valores (um por label):

```python
print(outputs.logits.shape)
torch.Size([2, 2])
```

Como temos apenas duas sentenças e duas labels, o resultado que obtemos de nosso modelo é de tamanho 2 x 2.

### Etapa de pós processamento
Os valores que obtemos como resultado de nosso modelo não fazem necessariamente sentido sozinhos. Vamos dar uma olhada:

```python
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)
```
Nosso modelo previu [-1.5607, 1.6123] para a primeira frase e [ 4.1692, -3.3464] para a segunda. Essas não são probabilidades, mas **logits**, a pontuação bruta e não normalizada produzida pela última camada do modelo. Para serem convertidos em probabilidades, eles precisam passar por uma camada **SoftMax** (todas saídas dos 🤗 Transformers produzem **logits**, já que a **função de loss (perda)** para treinamento geralmente fundirá a última função de ativação, como **SoftMax**, com a função de **loss real**, por exemplo a **cross entropy**), ou seja, a camada em que é aplicado o SoftMax é uma camada "pré-ativação":

```python
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)
```
Agora podemos ver que o modelo previu [0.0402, 0.9598] para a primeira frase e [0.9995, 0.0005] para a segunda. Estas são pontuações de probabilidade reconhecíveis.

Para obter as etiquetas correspondentes a cada posição, podemos inspecionar o atributo id2label da configuração do modelo (mais sobre isso na próxima seção):

```python
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}
```

Agora podemos concluir que o modelo previu o seguinte:
- A primeira frase: NEGATIVE: 0.0402, POSITIVE: 0.9598
- Segunda frase: NEGATIVE: 0.9995, POSITIVE: 0.0005

## Modelos

A classe ```AutoModel``` e todas as classes filhas são na verdade simples wrapper sobre a grande variedade de modelos disponíveis na biblioteca. É um wrapper inteligente, pois pode automaticamente “adivinhar” a arquitetura apropriada do modelo para seu checkpoint, e então instancia um modelo com esta arquitetura.

Entretanto, se você conhece o tipo de modelo que deseja usar, pode usar diretamente a classe que define sua arquitetura. Vamos dar uma olhada em como isto funciona com um modelo BERT.

### Criando um modelo
A primeira coisa que precisamos fazer para inicializar um modelo BERT é carregar um objeto de configuração:

```python
from transformers import BertConfig, BertModel

# Construindo a configuração
config = BertConfig()

# Construindo o modelo a partir da configuração
model = BertModel(config)
```

A configuração contém muitos atributos que são usados para construir o modelo:

```python
print(config)


BertConfig {
  [...]
  "hidden_size": 768,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  [...]
}
```

### Diferentes métodos de inicializar o modelo

A criação de um modelo a partir da configuração padrão o inicializa com valores aleatórios:

```python
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# O modelo é inicializado aleatoriamente!

```

O modelo pode ser utilizado neste estado, mas produzirá saídas errôneas; ele precisa ser treinado primeiro. Poderíamos treinar o modelo a partir do zero na tarefa em mãos, mas isto exigiria muito recurso, tempo e muitos dados. Para evitar esforços desnecessários e duplicados, normalmente é possível compartilhar e reutilizar modelos que já foram treinados.

Carregar um Transformer já treinado é simples - podemos fazer isso utilizando o método ```from_pretrained()```:


```python
from transformers import BertModel
model = BertModel.from_pretrained("bert-base-cased")
```

Como você viu anteriormente, poderíamos substituir o BertModel pela classe equivalente ao AutoModel. Faremos isto de agora em diante, pois isto produz um código generalista a partir de um checkpoint; se seu código funciona para checkpoint, ele deve funcionar perfeitamente com outro. Isto se aplica mesmo que a arquitetura seja diferente, desde que o checkpoint tenha sido treinado para uma tarefa semelhante (por exemplo, uma tarefa de análise de sentimento).

No exemplo de código acima não utilizamos BertConfig, e em vez disso carregamos um modelo pré-treinado através do identificador bert-base-cased. **Este é um checkpoint do modelo** que foi treinado pelos próprios autores do BERT; você pode encontrar mais detalhes sobre ele em seu [model card](https://huggingface.co/bert-base-cased).

Este modelo agora é inicializado com todos os pesos do checkpoint. Ele pode ser usado diretamente para inferência sobre as tarefas nas quais foi treinado, e também pode ser fine-tuned (aperfeiçoado) em uma nova tarefa. Treinando com pesos pré-treinados e não do zero, podemos rapidamente alcançar bons resultados.

Os pesos foram baixados e armazenados em cache (logo, para as futuras chamadas do método ```from_pretrained()``` não será realizado o download novamente) em sua respectiva pasta, que tem como padrão o path ```~/.cache/huggingface/transformers```. Você pode personalizar sua pasta de cache definindo a variável de ambiente HF_HOME.

O identificador usado para carregar o modelo pode ser o identificador de qualquer modelo no Model Hub, desde que seja compatível com a arquitetura BERT. A lista completa dos checkpoints BERT disponíveis podem ser encontrada [aqui].https://huggingface.co/models?filter=bert).

### Métodos para salvar/armazenar o modelo
Salvar um modelo é tão fácil quanto carregar um - utilizamos o método save_pretrained(), que é análogo ao método ```from_pretrained()```:

```python
model.save_pretrained("path_no_seu_computador")

# Isto salva dois arquivos em seu disco:

ls path_no_seu_computador
config.json pytorch_model.bin

```

Se você der uma olhada no arquivo config.json, você reconhecerá os atributos necessários para construir a arquitetura modelo. Este arquivo também contém alguns metadados, como a origem do checkpoint e a versão 🤗 Transformers que você estava usando quando salvou o checkpoint pela última vez.

O arquivo pytorch_model.bin é conhecido como o **dicionário de estado**; ele contém todos os pesos do seu modelo. Os dois arquivos andam de mãos dadas; **a configuração é necessária para conhecer a arquitetura de seu modelo**, enquanto **os pesos do modelo são os parâmetros de seu modelo**.

### Usando um modelo de Transformer para inferência
Os Tokenizers podem se encarregar de lançar as entradas nos tensores da estrutura apropriada, mas para ajudá-lo a entender o que está acontecendo, vamos dar uma rápida olhada no que deve ser feito antes de enviar as entradas para o modelo.

Digamos que temos um par de sequências

```python
sequences = ["Hello!", "Cool.", "Nice!"]

```

O tokenizer os converte em índices de vocabulário que são normalmente chamados de IDs de entrada. Cada sequência é agora uma lista de números! A saída resultante é:


```python
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

```

Esta é uma lista de sequências codificadas: uma lista de listas. Os tensores só aceitam shapes (tamanhos) retangulares (pense em matrizes). Esta “matriz” já é de forma retangular, portanto, convertê-la em um tensor é fácil:

```python
import torch
model_inputs = torch.tensor(encoded_sequences)
```

### Usando os tensores como entradas para o modelo

Fazer uso dos tensores com o modelo é extremamente simples - chamamos apenas o modelo com os inputs:

```python
output = model(model_inputs)
```

Embora o modelo aceite muitos argumentos diferentes, apenas os IDs de entrada são necessários. 

## Tokenizers

Os Tokenizers cuidam da primeira e da última etapa do processamento, cuidando da conversão de texto para entradas numéricas para a rede neural, e da conversão de volta ao texto quando for necessário. Eles são um dos componentes centrais do pipeline da PNL. Eles têm um propósito: traduzir texto em dados que podem ser processados pelo modelo. Os modelos só podem processar números, portanto os tokenizers precisam converter nossas entradas de texto em dados numéricos. O objetivo dos tokenizers é encontrar a representação mais significativa - ou seja, a que faz mais sentido para o modelo - e, se possível, a menor representação.

Os tokernizers podem ser divididos em:
- Baseado em palavras (word-based):
- Baseado em caracteres (Character-based)
- Tokenização por sub-palavras (Subword tokenization)


### Baseado em palavras (word-based):
O primeiro tipo de tokenizer que me vem à mente é baseado em palavras. É geralmente muito fácil de instalar e usar com apenas algumas regras, e muitas vezes produz resultados decentes. Por exemplo, na imagem abaixo, o objetivo é dividir o texto bruto em palavras e encontrar uma representação numérica para cada uma delas:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg">

Há diferentes maneiras de dividir o texto. Por exemplo, poderíamos utilizar o espaço em branco para simbolizar o texto em palavras, usando a função split() do Python:

```python
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)
['Jim', 'Henson', 'was', 'a', 'puppeteer']

```

Há também variações de tokenizers de palavras que têm regras extras para pontuação. Com este tipo de tokenizer, podemos terminar com alguns “vocabulários” bem grandes, onde um vocabulário é definido pelo número total de tokens independentes que tem no texto de exemplo.

A cada palavra é atribuída uma identificação, começando em 0 e indo até o tamanho do vocabulário. O modelo utiliza estas identificações para identificar cada palavra.

Se quisermos cobrir completamente um idioma com um tokenizer baseado em palavras, precisaremos ter um identificador para cada palavra no idioma, o que gerará uma enorme quantidade de tokens. Por exemplo, existem mais de 500.000 palavras no idioma inglês, portanto, **para construir um mapa a partir de cada palavra para um ID de entrada, precisaríamos manter um registro desse grande número de IDs**. Além disso, palavras como “dog” são representadas de forma diferente de palavras como “dogs”, e **o modelo inicialmente não terá como saber que “dog” e “dogs” são semelhantes**: ele identificará as **duas palavras como não relacionadas**. O mesmo se aplica a outras palavras semelhantes, como “run” e “running”, que o modelo não verá inicialmente como sendo semelhantes.

Finalmente, precisamos de token personalizada para representar palavras que não estão em nosso vocabulário. Isto é conhecido como o símbolo “unknown” (desconhecido), frequentemente representado como ”[UNK]” ou ””. Geralmente é um mau sinal se você vê que o tokenizer está produzindo muitos desses tokens, pois não foi capaz de recuperar uma representação sensata de uma palavra e você está perdendo informações ao longo do caminho. O objetivo ao elaborar o vocabulário é fazê-lo de tal forma que o tokenizer transforme o menor número possível de palavras no token desconhecido.

Principais pontos fracos:
- Vocabulários muito grandes
- Muitas palavras desconhecidas, por nao terem sido incluídas no dicionario
- Perda de significado entre palavras muito similares

Uma maneira de **reduzir a quantidade de tokens desconhecidas** é ir um nível mais fundo, usando um tokenizer baseado em caracteres.


### Baseado em caracteres (Character-based)
Os tokenizers baseados em caracteres dividem o texto em caracteres, ao invés de palavras. Isto tem dois benefícios principais:

O vocabulário será muito menor;
Há muito menos tokes fora de vocabulário (desconhecidas), uma vez que cada palavra pode ser construída a partir de personagens.
Mas também aqui surgem algumas questões sobre os espaços e à pontuação:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg">

Esta abordagem também não é perfeita. Como a representação agora é baseada em caracteres e não em palavras, pode-se argumentar que, intuitivamente, **ela é menos significativa**: **cada caractere não significa muito por si só**, ao contrario do caso das palavras. No entanto, isto novamente difere de acordo com o idioma; em chinês, por exemplo, cada caractere traz mais informações do que um caractere em um idioma latino.

Outra coisa a considerar é que **acabaremos com uma quantidade muito grande de tokens a serem processadas por nosso modelo**: enquanto uma palavra seria apenas um único token com um tokenizer baseado em palavras, ela pode facilmente se transformar em 10 ou mais tokens quando convertida em caracteres.

Principais pontos fracos:
- Sequências muito grandes
- Perda de significado entre tokens individuais

Para obter o **melhor dos dois mundos**, podemos usar uma terceira técnica que combina as duas abordagens: Tokenização por sub-palavras.


### Tokenização por sub-palavras (Subword tokenization)

Algoritmos de tokenização de sub-palavras baseiam-se no princípio de que **palavras frequentemente usadas não devem ser divididas em sub-palavras menores, mas palavras raras devem ser decompostas em sub-palavras significativas.**

Por exemplo, “irritantemente” poderia ser considerado uma palavra rara e poderia ser decomposto em “irritante” e “mente”. É provável que ambas apareçam mais frequentemente como sub-palavras isoladas, enquanto ao mesmo tempo o significado de “irritantemente” é mantido pelo significado composto de “irritante” e “mente”.

Aqui está um exemplo que mostra como um algoritmo de tokenização de uma sub-palavra indicaria a sequência “Let’s do tokenization!

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg">

Estas sub-palavras acabam fornecendo muito significado semântico: por exemplo, **no exemplo acima “tokenization” foi dividido em “token” e “ization”, dois tokens que têm um significado semântico enquanto são eficientes em termos de espaço** (apenas dois tokens são necessários para representar uma palavra longa). Isto **nos permite ter uma cobertura relativamente boa com pequenos vocabulários, e perto de nenhum token desconhecido.**

Esta abordagem é especialmente útil em idiomas aglutinativos como o turco, onde é possível formar palavras (quase) arbitrariamente longas e complexas, encadeando sub-palavras.


### Carregando e salvando
Carregando e salvando tokenizers é tão simples quanto com os modelos. Na verdade, ele se baseia nos mesmos dois métodos: ```from_pretrained()``` e ```save_pretrained()```. Estes métodos irão carregar ou salvar o algoritmo utilizado pelo tokenizer (um pouco como a ```arquitetura``` do modelo), bem como seu vocabulário (um pouco como os ```pesos``` do modelo).

O carregamento do tokenizer BERT treinado com o mesmo checkpoint do BERT é feito da mesma forma que o carregamento do modelo, exceto que utilizamos a classe ```BertTokenizer```:

```python
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
```

Similar ao AutoModel, a classe AutoTokenizer ira carregar a classe tokenizer apropriada na biblioteca com base no nome do checkpoint, e pode ser utilizada diretamente com qualquer checkpoint:

```python
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

```

Agora podemos usar o tokenizer, como mostrado na seção anterior:
```python
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 11303, 1200, 2443, 1110, 3014, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
```

Salvar um tokenizer é idêntico a salvar um modelo:
```python
tokenizer.save_pretrained("directory_on_my_computer")

output:
    ('directory_on_my_computer/tokenizer_config.json',
     'directory_on_my_computer/special_tokens_map.json',
     'directory_on_my_computer/vocab.txt',
     'directory_on_my_computer/added_tokens.json',
     'directory_on_my_computer/tokenizer.json')
```

### Encoding
Traduzir texto para números é conhecido como encoding. O encoding é feito em um processo de duas etapas: a tokenização, seguida pela conversão para IDs de entrada.

O primeiro passo é dividir o texto em palavras (ou partes de palavras, símbolos de pontuação, etc.), normalmente chamadas de tokens. Há várias regras que podem guiar esse processo, e é por isso que precisamos instanciar o tokenizer usando o nome do modelo, para nos certificarmos de usar as mesmas regras que foram usadas quando o modelo foi pré-treinado.

O segundo passo é converter esses tokens em números, para que possamos construir um tensor a partir deles e alimentá-los com o modelo. Para isso, **o tokenizer tem um vocabulário (vocabulary)**, que é a parte que realizamos o download quando o instanciamos com o método ```from_pretrained()```. Mais uma vez, precisamos utilizar o mesmo vocabulário utilizado quando o modelo foi pré-treinado

Para entender melhor os dois passos, vamos explorá-los separadamente. Note que usaremos alguns métodos que executam partes da pipeline de tokenização separadamente para mostrar os resultados intermediários dessas etapas, mas na prática, você deve chamar o tokenizer diretamente em suas entradas


O processo de tokenization é feito através do método tokenize() do tokenizer:

```python
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)
output:
    ['Using', 'a', 'transform', '##er', 'network', 'is', 'simple']
```    
Este tokenizer é um tokenizer de sub-palavras: ele divide as palavras até obter tokens que podem ser representadas por seu vocabulário. É o caso aqui do “transformer”, que é dividido em dois tokens: “transform” e “##er”.

A conversão para IDs de entrada é feita pelo método de tokenização ```convert_tokens_to_ids()```:


```python
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
[7993, 170, 11303, 1200, 2443, 1110, 3014]

```
Estas saídas, uma vez convertidas no tensor com a estrutura apropriada, podem então ser usadas como entradas para um modelo como visto anteriormente.

```python
final_inputs = tokenizer.prepare_for_model(ids)
print(final_inputs["input_ids"])

output:
    [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102]

```


### Decoding
Decoding vai pela direção ao contrário: a partir de índices de vocabulário, queremos obter uma string. Isto pode ser feito com o método decode() da seguinte forma:

```python
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

output:
    Using a transformer network is simple
```

Observe que o método decode não apenas converte os índices em tokens, mas também agrupa os tokens que fizeram parte das mesmas palavras para produzir uma frase legível. Este comportamento será extremamente útil quando utilizamos modelos que preveem um novo texto (seja texto gerado a partir de um prompt, ou para problemas de sequence-to-sequence como tradução ou sumarização).

## Tratando Sequencias Múltiplas

Os modelos esperam um batch de entradas. Anteriormente, vimos como as sequências são traduzidas em listas de números. Vamos converter esta lista de números em um tensor e enviá-la para o modelo

```python
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)
```

O problema é que enviamos uma única sequência para o modelo, enquanto que os 🤗 transformers esperam várias sentenças por padrão. Aqui tentamos fazer tudo o que o tokenizer fez nos bastidores quando o aplicamos a uma sequência, mas se você olhar com atenção, verá que ele não apenas converteu a lista de IDs de entrada em um tensor, mas acrescentou uma dimensão em cima dele:

```python
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])
```


Vamos tentar novamente e acrescentar uma nova dimensão:
```python
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids]) # IDS COMO LISTA AQUI
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

# Printamos os IDs de entrada assim como os logits resultantes - aqui está a saída:
Input IDs: [[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607, 2026,  2878,  2166,  1012]]
Logits: [[-2.7276,  2.8789]]
    
# Para termos a previsão, precisamos aplicar a função de ativação nos logits:
logits = output.logits
probabilities = F.softmax(logits, dim=1)
print("Probabilities:", probabilities)

predicted_class = torch.argmax(probabilities, dim=1)
print("Predicted class:", predicted_class.item())
    
``` 

***Batching*** é o ato de enviar múltiplas sentenças através do modelo, todas de uma só vez. Se você tiver apenas uma frase, você pode apenas construir um lote com uma única sequência: batched_ids = [ids, ids]

O Batching permite que o modelo funcione quando você o alimenta com várias frases. Usar várias sequências é tão simples quanto construir um lote com uma única sequência. Há uma segunda questão, no entanto. Quando você está tentando agrupar duas (ou mais) sentenças, elas podem ser de comprimentos diferentes. Se você já trabalhou com tensores antes, você sabe que eles precisam ser de forma retangular, então você não será capaz de converter a lista de IDs de entrada em um tensor diretamente. Para contornar este problema, normalmente realizamos uma **padronização (padding)** nas entradas. A seguinte lista de listas não pode ser convertida em um tensor:

```python
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]
```
O padding do ID token pode ser encontrada em tokenizer.pad_token_id. Vamos utilizá-lo e enviar nossas duas frases através do modelo individualmente e agrupadas em batches:

```python
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)
tensor([[ 1.5694, -1.3895],
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward>)
```
a segunda fileira deveria ser a mesma que os logits para a segunda frase, mas temos valores completamente diferentes!

Isto porque a característica chave dos Transformer são as camadas de atenção que contextualizam cada token. Estes levarão em conta os tokens de padding, uma vez que atendem a todos os tokens de uma sequência. Para obter o mesmo resultado ao passar frases individuais de diferentes comprimentos pelo modelo ou ao passar um batch com as mesmas frases e os paddings aplicados, precisamos dizer a essas camadas de atenção para ignorar os tokens de padding. Isto é feito com o uso de uma **máscara de atenção (attention mask)**.

**Attention masks** são tensores com a mesma forma exata do tensor de IDs de entrada, preenchidos com 0s e 1s: 1s indicam que os tokens correspondentes devem ser atendidas, e 0s indicam que os tokens correspondentes não devem ser atendidas (ou seja, devem ser ignoradas pelas camadas de atenção do modelo).

Vamos completar o exemplo anterior com uma máscara de atenção:

```python
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)
```

Agora obtemos os mesmos logits para a segunda frase do batch. Observe como o último valor da segunda sequência é um ID de padding, que é um valor 0 na máscara de atenção.

Com os Transformer, há um limite para os comprimentos das sequências, podemos passar os modelos. A maioria dos modelos manipula sequências de até 512 ou 1024 tokens, e se chocará quando solicitados a processar sequências mais longas. Há duas soluções para este problema:

- Use um modelo com suporte a um comprimento mais longo de sequência.
Trunque suas sequências.
- Os modelos têm diferentes comprimentos de sequência suportados, e alguns são especializados no tratamento de sequências muito longas. O Longformer é um exemplo, e outro exemplo é o LED. Se você estiver trabalhando em uma tarefa que requer sequências muito longas, recomendamos que você dê uma olhada nesses modelos.

Caso contrário, recomendamos que você trunque suas sequências, especificando o parâmetro max_sequence_length:

```python
sequence = sequence[:max_sequence_length]
```

## Agrupando tudo

```python
# Importando a biblioteca PyTorch e as classes necessárias do Transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Especificando o modelo pré-treinado a ser utilizado
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

# Inicializando o tokenizador associado ao modelo pré-treinado
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Inicializando o modelo pré-treinado para classificação de sequências
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Definindo duas sequências de texto para classificação
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Tokenizando as sequências e obtendo os logits
tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
logits = output.logits

# Aplicando softmax para obter probabilidades
from torch.nn.functional import softmax
probabilities = softmax(logits, dim=1)

# Determinando as classes previstas
predicted_classes = torch.argmax(probabilities, dim=1)

# Interpretando as classes previstas
class_labels = ["Negativo", "Positivo"]
for i, sequence in enumerate(sequences):
    print(f"Sequência: {sequence}")
    print(f"Classe Prevista: {class_labels[predicted_classes[i].item()]}")
    print(f"Probabilidades (Negativo, Positivo): {probabilities[i].tolist()}")
    print()

```

Este código usa um modelo pré-treinado (DistilBERT) finetunado para a tarefa de classificação binária no conjunto de dados SST-2 (sentiment analysis). Ele tokeniza duas sequências de texto, passa-as pelo modelo, calcula as probabilidades de pertencer a cada classe usando a função softmax e interpreta as classes previstas, imprimindo as sequências, as classes previstas e as probabilidades associadas.

# Fine tuning

## Préprocessando dados

### Exemplo
```python
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Carregando o modelo BERT pré-treinado e o tokenizador associado
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Definindo duas sequências de texto para classificação
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]

# Tokenizando as sequências, aplicando padding e truncamento, e retornando tensores PyTorch
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# Adicionando rótulos de classe ao lote (labels)
batch["labels"] = torch.tensor([1, 1])

# Inicializando o otimizador AdamW para ajustar os parâmetros do modelo
optimizer = AdamW(model.parameters())

# Realizando o cálculo da perda (loss) para as sequências no lote
# O modelo é chamado com as sequências tokenizadas e os rótulos (labels)
loss = model(**batch).loss

# Propagando o gradiente para realizar o ajuste dos parâmetros do modelo
loss.backward()

# Realizando uma etapa de otimização para atualizar os parâmetros
optimizer.step()
```

### Carregando um dataset

O hugginface hub contém diversos datasets, e com comandos simples podemos baixá-los em armazenar em cache. Por exemplo:

```python
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
```

Temos um ```DatasetDict```, que contém os dados de treino, validação e teste. Cada datapoint é composto por duas sentenças, e o label nos diz se as frases são paráfrases ou não, e o num_rows nos diz a quantidade de datapoints de cada conjunto de dados. Esse comando armazena o dataset em cache em ```~/.cache/huggingface/datasets```.

Podemos acessar os pares de frases no raw_datasets usando índices, como um dicionário

```python
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}
```

Para saber qual inteiro da label corresponde a qual label, podemos verificar as features do dataset ao executar o seguinte comando:

```python
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}
```

Atrás dos panos, o label é do tipo ClassLabel, e é mapeado em inteiros, em que 0 corresponde ao "not_equivalent" e 1 ao "equivalent".

### Preprocessando os datasets

Para preprocessar o dataset, precisamos converter o texto em representações numéricas para entrar como input no modelo. Fazemos isso utilizando o tokenizer, inputamos o tokenizer como uma sentença ou uma lista delas, e, podemos tokenizar todas diretamente. Como estamos lidando com pares de frases, precisamos tokenizar os pares.

```python
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{ 
  'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102],
  'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}
```
Aqui, o token_type_ids indica se o qual parte do input se refere a primeira sentença e qual se refere a segunda. Podemos decodificar de volta para palavras:

```python
tokenizer.convert_ids_to_tokens(inputs["input_ids"])
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']

# o modelo espera que os inputs estejam na forma de [CLS] sentence1 [SEP] sentence2 [SEP], alinhado com o que foi retornado no token_type_ids

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]
```

Outros checkpoints como o DistilBERT não vão retornar o token_type_ids, porquê ele não foi treinado com pares de frases como o BERT. 

Outra forma de tokenizar e de forma acelerada é através de uma função, como os datasets são arqiuvos de apache arrow, somente o valor acessado fica em memória. Esse processo pode ser feito com outros tipos de preprocessamento além da tokenização. Podemos usar a função através do método .map() e utilizando batched=True, aumentando a velocidade. Ao utilizar batched=True, a função é aplicada em multiplos elementos do dataset de uma vez, e não separadamente, aumentando a valocidade do preprocessamento

```python
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets
```
A forma como os datasets do huggingface aplicam o preprocessamento é adicionando novos campos ao dataset, um para cada chave no dicionário retornado pela função de preprocesamento.

```python
DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 408
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 1725
    })
})
```

Poderíamos também usar multiprocessamento ao aplicar sua função de pré-processamento com map() passando um argumento num_proc.

Nossa tokenize_function retorna um dicionário com as chaves input_ids, Attention_mask e token_type_ids, portanto, esses três campos são adicionados a todas as divisões do nosso conjunto de dados. Observe que também poderíamos ter alterado os campos existentes se nossa função de pré-processamento retornasse um novo valor para uma chave existente no conjunto de dados ao qual aplicamos map().

### Dynamic Padding

A função responsável por reunir amostras dentro de um lote é chamada de **collate function**. É um argumento que você pode passar ao construir um DataLoader, sendo o padrão uma função que apenas converterá suas amostras em tensores PyTorch e os concatenará (recursivamente se seus elementos forem listas, tuplas ou dicionários). Isso não será possível no nosso caso, pois as entradas que temos não serão todas do mesmo tamanho. Adiamos deliberadamente o preenchimento, para aplicá-lo apenas conforme necessário em cada lote e evitar entradas muito longas com muito preenchimento. Isso irá acelerar um pouco o treinamento, mas observe que se você estiver treinando em uma TPU, isso poderá causar problemas – as TPUs preferem formas fixas, mesmo quando isso requer preenchimento extra.

Para fazer isso na prática, temos que definir uma **collate function** que aplicará a quantidade correta de preenchimento aos itens do conjunto de dados que queremos agrupar em lote. Felizmente, a biblioteca 🤗 Transformers nos fornece essa função via **DataCollatorWithPadding**. É necessário um tokenizer quando você o instancia (para saber qual token de preenchimento usar e se o modelo espera que o preenchimento esteja à esquerda ou à direita das entradas) e fará tudo o que você precisa.

Para testar, vamos pegar algumas amostras do nosso conjunto de treinamento que gostaríamos de agrupar. Aqui, removemos as colunas idx, sentença1 e sentença2 porque elas não serão necessárias e contêm strings (e não podemos criar tensores com strings) e damos uma olhada nos comprimentos de cada entrada no lote:

```python
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]
```

Não é nenhuma surpresa, obtemos amostras de comprimentos variados, de 32 a 67. Preenchimento dinâmico significa que todas as amostras neste lote devem ser preenchidas até um comprimento de 67, o comprimento máximo dentro do lote. Sem preenchimento dinâmico, todas as amostras teriam que ser preenchidas até o comprimento máximo em todo o conjunto de dados ou até o comprimento máximo que o modelo pode aceitar. Vamos verificar se nosso data_collator está preenchendo dinamicamente o lote de maneira adequada:

```python
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 67]),
 'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'labels': torch.Size([8])}
```

## Finetune de modelo com o API Trainer ou Keras

### Training

🤗 Transformers oferece uma classe Trainer para ajudá-lo a ajustar qualquer um dos modelos pré-treinados fornecidos em seu conjunto de dados. Depois de concluir todo o trabalho de pré-processamento de dados na última seção, faltam apenas algumas etapas para definir a classe Trainer. A parte mais difícil provavelmente será preparar o ambiente para executar Trainer.train(), pois ele será executado muito lentamente em uma CPU.

Os exemplos de código abaixo pressupõem que você já executou os exemplos da seção anterior. Aqui está um breve resumo recapitulando o que você precisa:

```python
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

```

O primeiro passo antes de definirmos nosso Trainer é definir uma classe TrainingArguments que conterá todos os hiperparâmetros que o Trainer usará para treinamento e avaliação. O único argumento que você precisa fornecer é um diretório onde o modelo treinado será salvo, bem como os pontos de verificação ao longo do caminho. Para todo o resto, você pode deixar os padrões, o que deve funcionar muito bem para um ajuste básico.

```python
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

# define the trainer
training_args = TrainingArguments("test-trainer")

# defining the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
```

Você notará que, diferentemente do Capítulo 2, você receberá um aviso após instanciar este modelo pré-treinado. Isso ocorre porque o BERT não foi pré-treinado para classificar pares de sentenças, então o núcleo do modelo pré-treinado foi descartado e um novo núcleo adequado para classificação de sequência foi adicionado. Os avisos indicam que alguns pesos não foram utilizados (os correspondentes ao cabeçote de pré-treinamento descartado) e que outros foram inicializados aleatoriamente (os do novo cabeçote). Conclui incentivando você a treinar o modelo, que é exatamente o que faremos agora.

Assim que tivermos nosso modelo, podemos definir um Trainer passando para ele todos os objetos construídos até agora — o modelo, o training_args, os conjuntos de dados de treinamento e validação, nosso data_collator e nosso tokenizer:

```python
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# fine tuning
trainer.train()
```

Isso iniciará o ajuste fino (que deve levar alguns minutos em uma GPU) e relatará a perda de treinamento a cada 500 etapas. No entanto, não dirá quão bom (ou ruim) está o desempenho do seu modelo. Isto é porque:

- Não instruímos o instrutor a avaliar durante o treinamento, definindo a avaliação_estratégia como "etapas" (avaliar cada eval_steps) ou "época" (avaliar no final de cada época).
- Não fornecemos ao Trainer uma função compute_metrics() para calcular uma métrica durante a referida avaliação (caso contrário, a avaliação teria apenas impresso a perda, o que não é um número muito intuitivo).

### Evaluation

Vamos ver como podemos construir uma função computacional_metrics() útil e usá-la na próxima vez que treinarmos. A função deve pegar um objeto EvalPrediction (que é uma tupla nomeada com um campo de previsões e um campo label_ids) e retornará um dicionário de mapeamento de strings para floats (as strings são os nomes das métricas retornadas e os floats são seus valores). Para obter algumas previsões do nosso modelo, podemos usar o comando Trainer.predict():

```python
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
(408, 2) (408,)
```

A saída do método prever() é outra tupla nomeada com três campos: previsões, rótulo_ids e métricas. O campo de métricas conterá apenas a perda no conjunto de dados transmitido, bem como algumas métricas de tempo (quanto tempo levou para prever, no total e em média). Depois de concluirmos nossa função compute_metrics() e passá-la para o Trainer, esse campo também conterá as métricas retornadas por compute_metrics().

Como você pode ver, as previsões são uma matriz bidimensional com formato 408 x 2 (sendo 408 o número de elementos no conjunto de dados que usamos). Esses são os logits de cada elemento do conjunto de dados que passamos para predizer() (como você viu no capítulo anterior, todos os modelos do Transformer retornam logits). Para transformá-los em previsões que possamos comparar com nossos rótulos, precisamos pegar o índice com valor máximo no segundo eixo:

```python
import numpy as np
import evaluate

preds = np.argmax(predictions.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)
{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}
```

Os resultados exatos obtidos podem variar, pois a inicialização aleatória do cabeçote do modelo pode alterar as métricas alcançadas. Aqui, podemos ver que nosso modelo tem uma precisão de 85,78% no conjunto de validação e uma pontuação F1 de 89,97. Essas são as duas métricas usadas para avaliar os resultados do conjunto de dados MRPC para o benchmark GLUE. A tabela no artigo do BERT relatou uma pontuação F1 de 88,9 para o modelo básico. Esse foi o modelo sem case, enquanto atualmente usamos o modelo com case, o que explica o melhor resultado.

Juntando tudo, obtemos nossa função compute_metrics(). E para vê-lo usado em ação para relatar métricas no final de cada época, aqui está como definimos um novo Trainer com esta função compute_metrics():

```python
# Importando as bibliotecas e módulos necessários
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Definindo uma função para calcular métricas de avaliação
def compute_metrics(eval_preds):
    # Carregando a métrica específica do conjunto de dados MRPC do GLUE
    metric = load_metric("glue", "mrpc")

    # Obtendo as previsões e rótulos a partir das previsões do modelo
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Computando as métricas usando a métrica específica do GLUE para MRPC
    return metric.compute(predictions=predictions, references=labels)

# Configurando os argumentos de treinamento
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

# Inicializando o modelo para classificação de sequências com o número de rótulos (labels) definido como 2
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Inicializando o objeto Trainer para facilitar o treinamento
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Iniciando o treinamento fino (fine-tuning)
trainer.train()
```

Este código utiliza a biblioteca Transformers da Hugging Face para treinar um modelo de classificação de sequências (no caso, um modelo DistilBERT) no conjunto de dados MRPC do GLUE. Ele configura os argumentos de treinamento, inicializa o modelo, o tokenizador, e o objeto Trainer, e, finalmente, inicia o treinamento fino (fine-tuning) do modelo no conjunto de dados fornecido. A função compute_metrics é usada para calcular métricas de avaliação durante o treinamento.

Observe que criamos um novo TrainingArguments com sua ```evaluation_strategy``` definida como "época" e um novo modelo — caso contrário, estaríamos apenas continuando o treinamento do modelo que já treinamos. Para iniciar uma nova execução de treinamento, executamos ```trainer.train()```

Desta vez, ele reportará a perda de validação e as métricas no final de cada época, além da perda de treinamento. O Trainer funcionará imediatamente em várias GPUs ou TPUs e oferece muitas opções, como treinamento de precisão mista (use fp16 = True em seus argumentos de treinamento)

## A full training

Short summary...

```python
# Importando as bibliotecas necessárias
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Carregando o conjunto de dados MRPC do GLUE
raw_datasets = load_dataset("glue", "mrpc")

# Especificando o modelo pré-treinado que será utilizado (BERT neste caso)
checkpoint = "bert-base-uncased"

# Inicializando o tokenizador associado ao modelo pré-treinado
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Definindo uma função de tokenização que será aplicada ao conjunto de dados
def tokenize_function(example):
    # Aplicando a tokenização para cada exemplo no conjunto de dados
    # sentence1 e sentence2 são as chaves para as colunas de texto no conjunto de dados MRPC
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Aplicando a função de tokenização para cada exemplo no conjunto de dados, em lotes (batched)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Inicializando o objeto DataCollatorWithPadding, que será usado para processar os dados antes do treinamento
# Este objeto lida com o padding dos dados para que possam ser agrupados em lotes (batches)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
```

### Prepare for training

Antes de escrever nosso loop de treinamento, precisaremos definir alguns objetos. Os primeiros são os dataloaders que usaremos para iterar em lotes. Mas antes de podermos definir esses dataloaders, precisamos aplicar um pouco de pós-processamento aos nossos tokenized_datasets, para cuidar de algumas coisas que o Trainer fez automaticamente por nós. Especificamente, precisamos:

- Remova as colunas correspondentes aos valores que o modelo não espera (como as colunas sentença1 e sentença2).
- Renomeie o rótulo da coluna para rótulos (porque o modelo espera que o argumento seja nomeado rótulos).
- Defina o formato dos conjuntos de dados para que retornem tensores PyTorch em vez de listas.
Nosso tokenized_datasets possui um método para cada uma dessas etapas:

```python
# post processing
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

output:
["attention_mask", "input_ids", "labels", "token_type_ids"]

# criando o dataloader
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)ta_collator)
```

Para validar o processamento, podemos executar:

```python
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

output:
{'attention_mask': torch.Size([8, 65]),
 'input_ids': torch.Size([8, 65]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 65])}    
```

Agora que concluímos completamente o pré-processamento de dados , vamos voltar ao modelo. Nós o instanciamos exatamente como fizemos na seção anterior:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

output:
tensor(0.5441, grad_fn=<NllLossBackward>) torch.Size([8, 2])    

```

Todos os modelos de 🤗 Transformers retornarão a perda quando os rótulos forem fornecidos, e também obteremos os logits (dois para cada entrada em nosso lote, portanto, um tensor de tamanho 8 x 2).

Para o loop de treinamento faltam duas coisas: um otimizador e um scheduler de taxa de aprendizagem. Como estamos tentando replicar manualmente o que o Trainer estava fazendo, usaremos os mesmos padrões. O otimizador usado pelo Trainer é o AdamW, que é igual ao Adam, mas com um toque especial para regularização da queda de peso

```python
# Importando a classe de otimizador AdamW do módulo transformers
from transformers import AdamW

# Inicializando um otimizador AdamW para ajustar os parâmetros do modelo
# model.parameters() obtém todos os parâmetros treináveis do modelo
# lr=5e-5 define a taxa de aprendizado (learning rate) para o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)  # lr = taxa de aprendizado

# Importando a função get_scheduler do módulo transformers
from transformers import get_scheduler

# Definindo o número de épocas para treinamento
num_epochs = 3

# Calculando o número total de passos de treinamento
num_training_steps = num_epochs * len(train_dataloader)

# Inicializando um escalonador de taxa de aprendizado (learning rate)
# O tipo de escalonador é "linear", que ajusta linearmente a taxa de aprendizado durante o treinamento
# num_warmup_steps=0 significa que não há warm-up (um período inicial com taxa de aprendizado baixa)
# num_training_steps define o número total de passos de treinamento
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Imprimindo o número total de passos de treinamento
print(num_training_steps)


output:
1377
```

Neste código, um otimizador AdamW é inicializado para ajustar os parâmetros do modelo durante o treinamento. Além disso, um escalonador de taxa de aprendizado (learning rate) é configurado para ajustar linearmente a taxa de aprendizado ao longo dos passos de treinamento. O número total de passos de treinamento é calculado com base no número de épocas e no tamanho do dataloader de treinamento. O resultado é impresso como saída, indicando o número total de passos de treinamento que serão executados durante o treinamento do modelo.

### The training loop

Verificando se a GPU está disponível, caso sim, será usada no treinamento:

```Python
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

output:
device(type='cuda')    
```

Criando o loop de treinamento:
```python
# Importando a classe tqdm do módulo tqdm para exibir barras de progresso
from tqdm.auto import tqdm

# Inicializando uma barra de progresso com o número total de passos de treinamento
progress_bar = tqdm(range(num_training_steps))

# Colocando o modelo no modo de treinamento
model.train()

# Iterando sobre o número de épocas especificado para o treinamento
for epoch in range(num_epochs):
    # Iterando sobre os lotes (batches) de dados no dataloader de treinamento
    for batch in train_dataloader:
        # Movendo os dados para o dispositivo de treinamento (por exemplo, GPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Passando o lote pelo modelo e obtendo as saídas
        outputs = model(**batch)

        # Calculando a perda (loss) com base nas saídas do modelo
        loss = outputs.loss

        # Propagando o gradiente para realizar o ajuste dos parâmetros do modelo
        loss.backward()

        # Atualizando os parâmetros do modelo usando o otimizador
        optimizer.step()

        # Atualizando a taxa de aprendizado com o escalonador de taxa de aprendizado
        lr_scheduler.step()

        # Zerando os gradientes acumulados no otimizador
        optimizer.zero_grad()

        # Atualizando a barra de progresso para indicar o progresso do treinamento
        progress_bar.update(1)
```

Ele itera sobre o número especificado de épocas, passa por cada lote de dados no dataloader de treinamento, realiza a propagação para frente (forward pass) através do modelo, calcula a perda, realiza a retropropagação (backward pass) para calcular gradientes, atualiza os parâmetros do modelo usando o otimizador, e repete esse processo até que todas as épocas e passos de treinamento sejam concluídos. A barra de progresso é usada para visualizar o progresso do treinamento.

### Evaluation loop

Como fizemos anteriormente, usaremos uma métrica fornecida pela biblioteca 🤗 Evaluate. Já vimos o método metric.compute(), mas as métricas podem, na verdade, acumular lotes para nós à medida que percorremos o loop de previsão com o método add_batch(). Depois de acumularmos todos os lotes, podemos obter o resultado final com metric.compute(). 

```python
# Importando a biblioteca PyTorch e a função load do módulo evaluate
import torch
import evaluate

# Carregando a métrica específica do conjunto de dados MRPC do GLUE
metric = evaluate.load("glue", "mrpc")

# Colocando o modelo no modo de avaliação (evaluation)
model.eval()

# Iterando sobre os lotes (batches) de dados no dataloader de avaliação (eval_dataloader)
for batch in eval_dataloader:
    # Movendo os dados para o dispositivo de avaliação (por exemplo, GPU)
    batch = {k: v.to(device) for k, v in batch.items()}

    # Desativando o cálculo do gradiente durante a avaliação para economizar recursos
    with torch.no_grad():
        # Passando o lote pelo modelo e obtendo as saídas
        outputs = model(**batch)

    # Obtendo os logits do modelo
    logits = outputs.logits

    # Obtendo as previsões ao selecionar a classe com a maior probabilidade (argmax)
    predictions = torch.argmax(logits, dim=-1)

    # Adicionando o lote às estatísticas da métrica
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Calculando e imprimindo as métricas finais
metric.compute()

output:
{'accuracy': 0.8431372549019608, 'f1': 0.8907849829351535}
```

Este código avalia um modelo treinado no conjunto de dados MRPC do GLUE. Ele coloca o modelo no modo de avaliação (`model.eval()`), percorre os lotes de dados no dataloader de avaliação (`eval_dataloader`), passa cada lote pelo modelo, coleta as previsões, adiciona o lote às estatísticas da métrica e, finalmente, calcula e imprime as métricas finais usando a função `compute()` da métrica específica do conjunto de dados MRPC do GLUE.


### Agrupando tudo com o Train API

```python
Aqui está um exemplo comentado e explicado de preparação, treinamento em loop e validação em loop de fine-tuning usando a API Trainer da Hugging Face:

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Carregando o tokenizador e o modelo pré-treinado
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Tokenizando os conjuntos de treinamento e validação
train_tokenized = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
val_tokenized = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")

# Configurando os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

# Inicializando o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    tokenizer=tokenizer,
)

# Fine-tuning do modelo
trainer.train()

# Avaliação do modelo no conjunto de validação
results = trainer.evaluate()

# Imprimindo as métricas de avaliação
print("Results:", results)
```

Explicação:

1. **Carregando Tokenizador e Modelo Pré-Treinado:**
   - Um tokenizador e um modelo pré-treinado (DistilBERT neste exemplo) são carregados usando a API Transformers.

2. **Tokenização dos Dados de Treinamento e Validação:**
   - Os conjuntos de treinamento e validação são tokenizados usando o tokenizador carregado.

3. **Configuração dos Argumentos de Treinamento:**
   - Os argumentos de treinamento são configurados usando a classe `TrainingArguments`. Neste exemplo, o diretório de saída, a estratégia de avaliação, e o número de épocas de treinamento são definidos.

4. **Inicialização do Objeto Trainer:**
   - O objeto `Trainer` é inicializado com o modelo, os argumentos de treinamento, os conjuntos de treinamento e validação tokenizados, o data collator (para lidar com padding), e o tokenizador.

5. **Fine-tuning do Modelo:**
   - O método `train` do objeto `Trainer` é chamado para realizar o fine-tuning do modelo nos dados de treinamento.

6. **Avaliação do Modelo no Conjunto de Validação:**
   - O método `evaluate` do objeto `Trainer` é chamado para avaliar o modelo no conjunto de validação.

7. **Impressão das Métricas de Avaliação:**
   - As métricas de avaliação são impressas para avaliar o desempenho do modelo.

Agora, em loop

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch

# Exemplo de dados de treinamento e validação (substitua pelos seus próprios dados)
train_texts = ["I love fine-tuning!", "Fine-tuning is great!"]
val_texts = ["Fine-tuning is working.", "Hugging Face is awesome."]

# Carregando o tokenizador e o modelo pré-treinado
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Tokenizando os conjuntos de treinamento e validação
train_tokenized = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
val_tokenized = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")

# Configurando os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

# Inicializando o objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    tokenizer=tokenizer,
)

# Loop de treinamento
for epoch in range(training_args.num_train_epochs):
    # Treinamento
    trainer.train()

    # Avaliação
    results = trainer.evaluate()

    # Imprimir métricas de avaliação
    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs}")
    print("Results:", results)
    
    # Avaliação no conjunto de validação
    val_results = trainer.evaluate(eval_dataset=val_tokenized)
    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs} - Validation Results:", val_results)

# Salvar o modelo treinado
trainer.save_model("./final_model")
```

### Supercharge the trainig loop with huggingface Accelerate

O loop de treinamento que definimos anteriormente funciona bem em uma única CPU ou GPU. Mas usando a biblioteca 🤗 Accelerate, com apenas alguns ajustes podemos habilitar o treinamento distribuído em múltiplas GPUs ou TPUs. 

Aqui, o mesmo código mas modificado par utilizar o Accelerate:

```python
+ from accelerate import Accelerator
  from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

+ accelerator = Accelerator()

  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
  optimizer = AdamW(model.parameters(), lr=3e-5)

- device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
- model.to(device)

+ train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
+     train_dataloader, eval_dataloader, model, optimizer
+ )

  num_epochs = 3
  num_training_steps = num_epochs * len(train_dataloader)
  lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps
  )

  progress_bar = tqdm(range(num_training_steps))

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dataloader:
-         batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(**batch)
          loss = outputs.loss
-         loss.backward()
+         accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
        
```

A primeira linha a ser adicionada é a linha de importação. A segunda linha instancia um objeto Accelerator que examinará o ambiente e inicializará a configuração distribuída adequada. 🤗 O Accelerate cuida do posicionamento do dispositivo para você, para que você possa remover as linhas que colocam o modelo no dispositivo (ou, se preferir, alterá-las para usar accelerator.device em vez de device).

Em seguida, a maior parte do trabalho é realizada na linha que envia os dataloaders, o modelo e o otimizador para accelerator.prepare(). Isso colocará esses objetos no contêiner adequado para garantir que seu treinamento distribuído funcione conforme planejado. As alterações restantes a serem feitas são remover a linha que coloca o lote no dispositivo (novamente, se você quiser manter isso, basta alterá-la para usar accelerator.device) e substituir loss.backward() por accelerator.backward(loss).

Colocar isso em um script train.py tornará esse script executável em qualquer tipo de configuração distribuída. Para experimentar em sua configuração distribuída, execute o comando:

```python
# solicitará que você responda algumas perguntas e coloque suas respostas em um arquivo de configuração usado por este comando
accelerate config

# lançará o treinamento distribuído.
accelerate launch train.py
```

# Compartilhando Modelos e Tokenizers

# Datasets Library

# Principais tasks de NLP

# Como pedir ajuda

# Desenvolvendo e Compartilhando Demos